In [50]:
%run titanic_2_data.ipynb

install_package('scikit-learn')

Note: you may need to restart the kernel to use updated packages.
snowflake-connector-python has been installed.


/tmp/ipykernel_15467/2994179729.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


Note: you may need to restart the kernel to use updated packages.
scikit-learn has been installed.


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

### Data preprocessing

In [52]:
# Select the features to be used for prediction
features = ['PCLASS', 'SEX', 'AGE', 'SIBSP', 'PARCH']
target = 'SURVIVED'

In [53]:
# Remove unnecessary columns
data = data.drop(['NAME', 'TICKET', 'FARE', 'CABIN', 'EMBARKED', 'BOAT', 'BODY', 'HOME_DEST'], axis=1)

# Convert categorical variables to numerical
data['SEX'] = data['SEX'].map({'male': 0, 'female': 1})

### Split the dataset into features and target

In [54]:
# Handle missing values
data[features] = data[features].fillna(data[features].mean())

data[target] = data[target].fillna(data[target].mean()).astype(int)

Method for training the data

In [55]:
def train(X, Y, model, train_size):
    res = {}
    x_train_subset, x_test_subset, y_train_subset, y_test_subset = train_test_split(X, Y, train_size=train_size, random_state=42)
    model.fit(x_train_subset, y_train_subset)
    y_pred_train_subset = model.predict(x_train_subset)
    y_pred_test = model.predict(x_test_subset)
    res['train_accuracy'] = accuracy_score(y_train_subset, y_pred_train_subset)
    res['test_accuracy'] = accuracy_score(y_test_subset, y_pred_test)
    y_pred_proba = model.predict_proba(x_test_subset)[:, 1]
    res['loss'] = log_loss(y_test_subset, y_pred_proba)
    return res

Utils methods to initialize needed curves data

In [56]:
def generate_curve_data(X, Y, model, train_sizes):
    train_accuracy_curve = []
    test_accuracy_curve = []
    loss_curve = []
    for train_size in train_sizes:
        res = train(X, Y, model, train_size)
        train_accuracy_curve.append(res['train_accuracy'])
        test_accuracy_curve.append(res['test_accuracy'])
        loss_curve.append(res['loss'])
    return {
        'train_accuracy': train_accuracy_curve,
        'test_accuracy': test_accuracy_curve,
        'loss': loss_curve,
    }